## Notes

Diminuer le pas, va marcher mais trop long
gradient amorti, gradient à pas optimal

c1 = 10^-4, c2 = 0.9

Style de rendu ?

Questions
- q6 manque une composante vecteur c, alpha non négligeable ?
- q7 max coût ou profit ?
- q8 norme non différentiable en 0 ?
- calculer gradient à la main ou autograd ? besoin pour scipy ? problème décompose vecteur z...
- deux algorithmes à trouver... : newton avec pénalisation (mais pas dans le cours ?) et gradient proximal ?
- vérifier avec scipy (ou à utiliser ?), quel choix de méthode ? calcul gradient des fonctions & des coûts ?
- scipy à la fin : production négative : rajouter des contraintes
- résultat pas entier : moitié de croissant ? où arrondir ?
- résultat différent : mauvaise valeur de alpha... ? à quoi s'attendre ?
- grande variabilité selon la valeur de alpha ? bizarre première méthode... ou problème scipy slsqp sans gradient spécifié (important) ?


In [101]:
import numpy as np
from scipy import optimize
import casadi
from casadi import SX, mtimes, transpose, exp, sum1, fmin, MX
np.set_printoptions(suppress=True)

## 1. Etude du problème d'optimisation

### Question 1

Le coût (2) correspond au profit de la boulangerie :

- $c^T r$ est le coût d'achat des matières premières
- $v^T q$ est le montant rapporté par la vente d'une quantité de produits $q$ (tout ce qui a été produit)
- $v^T d$ est le montant rapporté par la vente d'une quantité de produits $d$ (tout ce qui a été demandé)

Ainsi, le terme $\min\{q,d\}$ correspond à la quantité minimum entre ce qui a été produit ou demandé. En effet, le facteur limitant des gains dépend si la demande est supérieure à la production ou si la production est supérieure à la demande (dans le cas d'économies d'échelle par exemple, cela peut être rentable).


### Question 2

Ce terme avec un $\min$ n'est pas un opérateur linéaire, il n'est donc peut-être pas différentiable et ne permet pas d'appliquer les méthodes du cours les plus efficaces.

### Question 3

Cette formule fait penser à la statistique de Maxwell-Boltzmann.

Soit $i \in [|1, p|]$.

Cas 1 : $q_i > d_i$. Montrons que $h_i \approx d_i$.

$\exp(-\alpha q_i) << \exp(-\alpha d_i)$ et $q_i \exp(-\alpha q_i)$ est toujours petit devant $d_i \exp(-\alpha d_i)$ car $\alpha >> 1$, ce qui permet de simplifier et d'obtenir le résultat.

Cas 2 : $q_i < d_i$. On procède de manière similaire par symétrie

On a donc maintenant une fonction convexe différentiable ce qui permet de se placer dans un cadre où l'on dispose plus de résultats.

### Question 4

On suppose que $d$ est fixé. On pose :

- $z = (q, r)$ (de taille $n=p + m$)
- $f(z) = f(q, r) = c^T r - v^T h(q, d)$, à valeurs dans $\mathbb{R}$
- $c(z) = c(q, r) = A q - r$ ($m$ contraintes)

## 2. Etude et résolution numérique

### Question 5

Puisque la fonction $f$ est deux-fois différentiable, on peut utiliser la méthode la plus efficace pour l'optimisation différentiable, c'est-à-dire la méthode de Newton. Toutefois, il faut prendre en compte les contraintes inégalités. On peut donc penser à une optimisation avec pénalités.

Une autre approche serait les algorithmes d'optimisiation différentiable sous contraintes, comme celui Uzawa ou d'Arrow-Hurwicz (les contraintes sont linéaires)

### Question 6

Il est surprenant que $1$ ne soit pas négligeable devant $\alpha$...

In [46]:

def q6():
    alpha = 0.1
    c = 1e-3*np.array([30.,1.,1.3,4., 1.])
    v = np.array([0.9, 1.5, 1.1])
    d = np.array([400., 67., 33.])
    A = np.array([[3.5,2.,1.],[250.,80., 25.],[0., 8., 3.],[0., 40., 10.], [0., 8.5, 0.]])
    n = len(c)+len(d)
    return n, alpha, c, v, d, A

n, alpha, c, v, d, A = q6()
x0 = np.zeros(n)
print(n)

8


In [47]:
def fun_hp(q, d, alpha=alpha):
    return (q*exp(-alpha*q)+d*exp(-alpha*d))/(exp(-alpha*q)+exp(-alpha*d))

In [58]:
k = len(d)

opti = casadi.Opti();

x = opti.variable(n)
q = x[0:k]
r = x[k:]
cp = opti.parameter(n-k)
vp = opti.parameter(k)
dp = opti.parameter(k)
ap = opti.parameter(n-k,k)

f = mtimes(transpose(cp), r) - mtimes(transpose(vp), fun_hp(q, dp))
opti.minimize(f)
opti.subject_to(mtimes(ap, q) - r <= 0)

x0 = np.array([0.]*n)
opti.set_initial(x,x0)
# pour enlever le blabla, définir un dictionnaire d'option
options = {'print_time':False, 'ipopt':{'print_level':0}}
# options = dict(print_time=False, ipopt=dict(print_level=0))
opti.solver('ipopt', options);


opti.set_value(cp, c)
opti.set_value(vp, v)
opti.set_value(dp, d)
opti.set_value(ap, A)

sol = opti.solve();
print(sol.value(x))

[   402.12704191     73.10746942     42.43483578   1596.09442137
 107441.22892767    712.1642646    3348.6471351     621.41349253]


In [45]:
def fun_h(q, d, alpha=alpha):
    return (q*np.exp(-alpha*q)+d*np.exp(-alpha*d))/(np.exp(-alpha*q)+np.exp(-alpha*d))

print(fun_h(np.array([1.,2.,3.]), d))

def fun_f(z, d=d, A=A, c=c, v=v, alpha=alpha):
    q = z[:len(d)]
    r = z[len(d):]
    return c.T @ r - v.T @ fun_h(q, d, alpha)

ztest = np.array([1.,2.,3.,4.,5.,6.,7.,8.])
print(fun_f(ztest))

def fun_c(z, A=A):
    q = z[:len(d)]
    r = z[len(d):]
    return A @ q - r

print(c)

[1.         2.09757685 4.4227762 ]
-8.742619084891563
[0.03   0.001  0.0013 0.004  0.001 ]


In [9]:
ineq_cons = { 'type' : 'ineq' , 'fun' : lambda x: - fun_c(x)}
res = optimize . minimize(fun_f, x0, method= 'SLSQP' ,
constraints= [ineq_cons], options={ 'disp' : True, 'maxiter': 10000})

print(res.x)

Optimization terminated successfully    (Exit mode 0)
            Current function value: -333.52157831268005
            Iterations: 63
            Function evaluations: 584
            Gradient evaluations: 63
[   402.12800137     73.10884775     42.43652509   1596.1022254
 107441.62129052    712.18035732   3348.71916112    621.42520591]


Légère surproduction : très étonnant, économie d'échelle ? pas tout à fait comme intuition : plutôt moins ou = à la demande mais pas plus...


Comparer aussi la valeur du profit en lui-même... ?

### Question 7

a. Problème d'optimisation.

On calcule l'espérance de l'opposé du profit : $E(z)=\pi_1 f_1(z)+\pi_2 f_2(z)+\pi_3 f_3(z)$ où $f_i(z)= c^T r - v^T h(q, d^i)$

On minimise donc cette quantité $E(z)$ avec les mêmes contraintes.



In [63]:
def q7():
    d1 = np.array([400., 67., 33.])
    d2 = np.array([500., 80., 53.])
    d3 = np.array([300., 60., 43.])
    p1 = 0.5
    p2 = 0.3
    p3 = 0.2
    return d1, p1, d2, p2, d3, p3

ps = q7()
d1, p1, d2, p2, d3, p3 = q7()


In [64]:
def fun_E(z, ps=ps):
    d1, p1, d2, p2, d3, p3 = ps
    return p1*fun_f(z, d1)+p2*fun_f(z, d2)+p3*fun_f(z, d3)

In [65]:
ineq_cons = { 'type' : 'ineq' , 'fun' : lambda x: - fun_c(x)}
res = optimize . minimize(fun_E, x0, method= 'SLSQP' ,
constraints= [ineq_cons], options={ 'disp' : True, 'maxiter': 10000})

print(res.x)

Optimization terminated successfully    (Exit mode 0)
            Current function value: -323.9598421618229
            Iterations: 67
            Function evaluations: 625
            Gradient evaluations: 67
[   406.70791452     77.13971119     54.28026736   1632.03739057
 109205.16221001    779.9584916    3628.39112119    655.68754512]


In [70]:

opti = casadi.Opti();

x = opti.variable(n)
q = x[0:k]
r = x[k:]
cp = opti.parameter(n-k)
vp = opti.parameter(k)
dp = opti.parameter(3, k)
pp = opti.parameter(k)
ap = opti.parameter(n-k,k)

f = 0
for j in range(3):
    f += pp[j]*(mtimes(transpose(cp), r) - mtimes(transpose(vp), fun_hp(q, transpose(dp[j,:]))))

opti.minimize(f)
opti.subject_to(mtimes(ap, q) - r <= 0)

x0 = np.array([0.]*n)
opti.set_initial(x,x0)
# pour enlever le blabla, définir un dictionnaire d'option
options = {'print_time':False, 'ipopt':{'print_level':0}}
# options = dict(print_time=False, ipopt=dict(print_level=0))
opti.solver('ipopt', options);


opti.set_value(cp, c)
opti.set_value(vp, v)
opti.set_value(ap, A)
opti.set_value(pp, [p1, p2, p3])
opti.set_value(dp, np.array([d1, d2, d3]))

sol = opti.solve();
print(sol.value(x))

[   406.69842673     77.13921035     54.28956029   1632.01247461
 109202.9825197     779.98236557   3628.46401742    655.68329044]


Produit un peu plus pour couvrir la variabilité si plus de demande car proba plus de demande plus importante que demande plus faible ?

Comparer aussi la valeur du profit en lui-même...

### Question 8

a. On suppose que $r$ et $d$ sont fixés. On a toujours le terme de revenus $v^T h(q,d)$. Le premier terme correspond à $(q-d)^T(q-d)$. $q-d$ est la différence entre ce qui est produit et ce qui est demandé. Cela correspond donc au coût induit par le gaspillage si $q>d$, ou le coût de la non-satisfaction de la demande si $d>q$d. On cherche donc bien à minimiser ce coût...

b. 

In [73]:
ineq_cons = { 'type' : 'ineq' , 'fun' : lambda x: - fun_c(x)}
res = optimize . minimize(fun_E, x0, method= 'SLSQP' ,
constraints= [ineq_cons], options={ 'disp' : True, 'maxiter': 10000})

y = res.x
#print(y)
r = y[len(d):]
#print(r)

def fun_g(q, d=ps[4], v=v):
    return (q-d).T @ (q-d) - v.T @ fun_h(q, d) # attention norme au carré

def fun_cb(q, r=r, d=ps[4]):
    return fun_c(np.hstack((q, r)))

y0 = np.zeros(len(d))
#print(y0)

ineq_cons = { 'type' : 'ineq' , 'fun' : lambda x: - fun_cb(x)}
res = optimize . minimize(fun_g, y0, method= 'SLSQP' ,
constraints= [ineq_cons], options={ 'disp' : True, 'maxiter': 10000})
print(res.x)




Optimization terminated successfully    (Exit mode 0)
            Current function value: -323.9598421618229
            Iterations: 67
            Function evaluations: 625
            Gradient evaluations: 67
Optimization terminated successfully    (Exit mode 0)
            Current function value: -407.55865445651534
            Iterations: 7
            Function evaluations: 30
            Gradient evaluations: 7
[300.21944431  60.36145645  43.26755046]


In [76]:

opti = casadi.Opti();

q = opti.variable(k)
cp = opti.parameter(n-k)
vp = opti.parameter(k)
dp = opti.parameter(k)
ap = opti.parameter(n-k,k)
rp = opti.parameter(n-k)

f = mtimes(transpose(q-dp), q-dp) - mtimes(transpose(vp), fun_hp(q, dp))
opti.minimize(f)
opti.subject_to(mtimes(ap, q) - rp <= 0)

x0 = np.array([0.]*k)
opti.set_initial(q,x0)
# pour enlever le blabla, définir un dictionnaire d'option
options = {'print_time':False, 'ipopt':{'print_level':0}}
# options = dict(print_time=False, ipopt=dict(print_level=0))
opti.solver('ipopt', options);


opti.set_value(cp, c)
opti.set_value(vp, v)
opti.set_value(dp, ps[4])
opti.set_value(ap, A)
opti.set_value(rp, r)

sol = opti.solve();
print(sol.value(q))

[300.22004929  60.36144863  43.26764076]


Moins de gaspillage ? Adapté Très proche de la demande $d^3$


Comparer aussi la valeur du profit en lui-même...

## 3. Etude du problème non-régularisé

### Question 9

Méthode de gradient proximale (optimisation non-différentiable ?) (cf Traitement du Signal ?)

### Question 10

In [14]:
def fun_hb(q, d, alpha=alpha):
    return np.minimum(q,d)

print(fun_hb(np.array([1.,2.,3.]), d))

def fun_fb(z, d=d, A=A, c=c, v=v, alpha=alpha):
    q = z[:len(d)]
    r = z[len(d):]
    return c.T @ r - v.T @ fun_hb(q, d, alpha)

def fun_cb(z, A=A):
    q = z[:len(d)]
    r = z[len(d):]
    return np.hstack((A @ q - r,-q))

ztest = np.array([1.,2.,3.,4.,5.,6.,7.,8.])
print(fun_fb(ztest))

[1. 2. 3.]
-7.0312


In [15]:
ineq_cons = { 'type' : 'ineq' , 'fun' : lambda x: - fun_cb(x)}
res = optimize . minimize(fun_fb, x0, method= 'SLSQP' ,
constraints= [ineq_cons], options={ 'disp' : True, 'maxiter': 10000})
print(res.x)

# résultat fantasiste : méthode de scipy pas adaptée au problème ? profit positif car opposé du résultat...
# SLSQP pourtant OK pour non linéaire ? problème gradient ?
# problème alpha ?

# pas possible production négative ?? rajout d'une contrainte : fun_cb

Optimization terminated successfully    (Exit mode 0)
            Current function value: -147.69824285037322
            Iterations: 81
            Function evaluations: 912
            Gradient evaluations: 81
[   399.99999944     66.11016455     41.28196352   1573.50229064
 106320.86211083  12525.58755043  42272.26996026   8868.72897784]


Résultat assez différents (quantifier ?) : normal ou non ?

In [96]:

def q6():
    alpha = 0.1
    c = 1e-3*np.array([30.,1.,1.3,4., 1.])
    v = np.array([0.9, 1.5, 1.1])
    d = np.array([400., 67., 33.])
    A = np.array([[3.5,2.,1.],[250.,80., 25.],[0., 8., 3.],[0., 40., 10.], [0., 8.5, 0.]])
    n = len(c)+len(d)
    return n, alpha, c, v, d, A

n, alpha, c, v, d, A = q6()
x0 = np.zeros(n)
print(n)

8


In [104]:
k = len(d)

opti = casadi.Opti();

x = opti.variable(n)
q = x[0:k]
r = x[k:]
cp = opti.parameter(n-k)
vp = opti.parameter(k)
dp = opti.parameter(k)
ap = opti.parameter(n-k,k)

def my_min(a, b):
    cz = SX(k)
    for j in range(k):
        cz[j] = (a[j] - b[j])*(a[j]-b[j])
    return cz
# au carré ?

# contrainte, q & r positives

f = mtimes(transpose(cp), r) - mtimes(transpose(vp), my_min(q,d))
opti.minimize(f)
opti.subject_to(mtimes(ap, q) - r <= 0)
opti.subject_to( - r <= 0)
opti.subject_to( - q <= 0)

x0 = np.array([0.]*n)
opti.set_initial(x,x0)
# pour enlever le blabla, définir un dictionnaire d'option
options = {'print_time':False, 'ipopt':{'print_level':0}}
# options = dict(print_time=False, ipopt=dict(print_level=0))
opti.solver('ipopt', options);


opti.set_value(cp, c)
opti.set_value(vp, v)
opti.set_value(dp, d)
opti.set_value(ap, A)

try:
    sol = opti.solve();
    print(sol.value(x))
except:
    print("erreur")
    print(opti.debug.value)

NotImplementedError: Wrong number or type of arguments for overloaded function 'SX_set'.
  Possible prototypes are:
    set(self,SX,bool,Sparsity)
    set(self,SX,bool,Slice)
    set(self,SX,bool,IM)
    set(self,SX,bool,Slice,Slice)
    set(self,SX,bool,Slice,IM)
    set(self,SX,bool,IM,Slice)
    set(self,SX,bool,IM,IM)
  You have: '(SX,MX,bool,int)'


min de l'espérance q7 attention
3 lignes  problème bien posé

min (q,d) 
probabilité



In [ ]:
convexe, non convexe, contraintes ou non, pas besoin d'être exhaustif
tp4 utile pour le miniprojet

In [ ]:
écrire le problème non différentielle
transformation du problème : rajouter des contraintes qui reproduise le problème là

problème équivalent : variable équivalent, toutes les variables est dériables

problème linéaire sauf le min
méthode basée dérivée

que pas contrainte solution car linéaire

gradient toujours le même

conditions de KKT

méthode basée sur la dérivée
méthode de gradient

régulariser : alpha q^2

toutes acceptables

imagination,
toutes les méthodes basées sur la dérivée vont échouée


sans le min: résoudre chaque problème


In [ ]:
Discussion avec le prof:
utiliser np.dot(q, q) pour norme de q au carré
mtimes vérifier pas pour mult element par element

q7 : min(q, r) de E_d(v^t*h()-...)
écrire ainsi

2 solutions pour q10 : gradient proximal (sous différentiel) : normaliser en rajoutant + alpha * np.dot(q, q) + alpha * np.dot(r,r) et faire tendre alpha vers 0... voir si cela se passe bien

OU décomposer avec contrainte
production = q + s avec s = slack (vendu à coût nul si production supérieur à la demande) et q vendu à coût non nul
contrainte  q <= d 
et pour que s soit nul si q <= d, contrainte de complémentarité s(q-d)=0
et enlever le min, à faire avec casadi

enfin, chercher un résultat entier (contrainte en plus pour casadi)
ou écrire que x = sum(n_i y_i) avec n_i = 1, 2, ... et y_i dans [0, 1] avec contrainte sum(y_i) <= 1 ???